# Basic model introduction

This page introduces the processes for building and running a simple compartmental disease model with _summer_.
In the following example, we will create an SIR compartmental model for a general, unspecified emerging infectious disease spreading through a fully susceptible population. In this model there will be:

- Three compartments: susceptible (S), infected (I) and recovered (R)
- A starting population of 1000 people, with 10 of them infected (and infectious)
- An evaluation timespan from day zero to 20 units of time (we can think of these as days, but the time unit is arbitrary as far as the model is concerned)
- Inter-compartmental flows for infection, deaths and recovery

Let's look at a complete example of this model in action, and then examine the details of each step.
This is the complete example model that we will be working with.
As with any Python script, we first have to import the objects we need,
plus we'll set the visualisation of pandas objects to be interactive.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict

from summer import CompartmentalModel
from summer.parameters import Parameter as param

pd.options.plotting.backend = "plotly"

First, let's create a function that gives us a base SIR model with the
basic compartments, starting populations and inter-compartmental flows implemented.

In [ ]:
def get_base_sir_model():
    """
    Generate an instance of an SIR model with some standard parameters, 
    population distribution and parameters hard-coded.
    """
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    analysis_times = (0, 20)
    
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=["infectious"],
    )
    model.set_initial_population(
        distribution={"susceptible": 990, "infectious": 10}
    )
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=1.,
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=0.333,
        source="infectious", 
        dest="recovered",
    )
    model.add_death_flow(
        name="infection_death", 
        death_rate=0.05,
        source="infectious",
    )
    return model

Then use the function to get an instance of this model,
run it and have a look at the compartment size progression over time.
Note that we use the plotting functions built-in to pandas objects to do this,
rather than writing any of our own plotting code.
Here it is set to use "plotly",
but this can be immediately changed to other backends by changing the
"plotly" option above (e.g. to "matplotlib").

In [ ]:
base_model = get_base_sir_model()  # Get the model object
base_model.run()  # Run it
compartment_values = base_model.get_outputs_df()  # Access the outputs as a pandas dataframe
compartment_values.plot()  # Plot

However, we can do this slightly better,
by asking the model object to _expect_ certain parameters,
which we define as parameter objects,
but don't give values yet.
We'll come back to why this is worth doing later,
but for now let's just say it is the preferred approach.
While we're at it, we'll also make the times and starting
population values part of the inputs to this function,
even though they go into the model object just as values.

In [ ]:
def get_param_aware_sir_model(
    parameters: Dict,
):
    """
    Generate an instance of an SIR model that is expecting parameter values to be provided
    for the transition rates and some other features.
    
    Args:
        parameters: The parameter values to be used in running the model    
    """
    
    # Define the 
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    infectious_compartment = [
        "infectious",
    ]
    analysis_times = (
        parameters["start_time"], 
        parameters["end_time"]
    )

    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=infectious_compartment,
        takes_params=True,
    )
    
    # Check and assign infectious seed
    pop = parameters["population"]
    seed = parameters["seed"]
    suscept_pop = pop - seed
    msg = "Seed larger than population"
    assert pop >= 0.
    
    model.set_initial_population(
        distribution={
            "susceptible": suscept_pop, 
            "infectious": seed}
    )
    
    # Add a frequency-dependent transmission flow
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=param("contact_rate"), 
        source="susceptible", 
        dest="infectious",
    )
    
    # Add a constant recovery flow
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=param("recovery_rate"), 
        source="infectious", 
        dest="recovered",
    )
    
    # Add a constant infection-related death rate
    model.add_death_flow(
        name="infection_death", 
        death_rate=param("death_rate"), 
        source="infectious",
    )
    return model

Now let's use that function, specifying our parameters all in one go
and feeding them into the model object when we're ready to run it.

In [ ]:
parameters = {
    "contact_rate": 1.,
    "recovery_rate": 0.333,
    "death_rate": 0.05,
    "population": 1000.,
    "seed": 10.,
    "start_time": 0.,
    "end_time": 20.,
}

param_aware_model = get_param_aware_sir_model(parameters)
param_aware_model.run(parameters=parameters)
compartment_values = param_aware_model.get_outputs_df()
compartment_values.plot()

Now that we have our [CompartmentalModel](/api/model.html) object,
we can use this structure to inspect some aspects of what is going on under the surface,
for example, compartments, flows and others attributes.
This is highly recommended, 
to ensure that the model you have created is consistent with what you were wanting.
Try out using tab complete in this notebook to inspect the range of methods and
attributes that are available for a _CompartmentalModel_ object.

In [ ]:
print(param_aware_model.compartments)
print(param_aware_model.initial_population)
print(param_aware_model.times)
print(compartment_values)

## Summary

That's it for now, now you know how to:

- Create a model
- Add a population
- Add flows
- Run the model
- Access and visualize the outputs

A detailed API reference for the CompartmentalModel class can be found [here](http://summerepi.com/api/model.html)

## Checking how the model works inside

This section presents some code that shows an approximation of what is happening inside the model we just built and ran.
This is a sanity check that the outputs look as we would expect if we had coded this manually,
and is intended to provide some insight into what is going on under the surface.

In the example code below we use the [Euler method](https://en.wikipedia.org/wiki/Euler_method) to solve an ordinary differential equation (ODE) which is defined by the model's flows. We don't typically use Euler in _summer_ though, you can read more about the actual ODE solvers available to evaluate models [here](http://summerepi.com/examples/4-ode-solvers.html).

In [ ]:
TIMESTEP = 0.1
START_TIME = 0
END_TIME = 20

# Get times
time_period = END_TIME - START_TIME + 1
num_steps = time_period / TIMESTEP
times = np.linspace(START_TIME, END_TIME, num=int(num_steps))

# Define initial conditions and prepare for outputs
initial_conditions = np.array(
    [
        parameters["population"] - parameters["seed"], 
        parameters["seed"],
        0.
    ]
)
outputs = np.zeros((int(num_steps), 3))
outputs[0] = initial_conditions

# Model parameters
contact_rate = parameters["contact_rate"]
recovery_rate = parameters["recovery_rate"]
death_rate = parameters["death_rate"]

# Calculate outputs for each timestep
for t_idx, t in enumerate(times):
    if t_idx == 0:
        continue

    # Get some useful quantities that we'll need later
    flow_rates = np.zeros(3)
    compartment_sizes = outputs[t_idx - 1]
    num_sus = compartment_sizes[0]
    num_inf = compartment_sizes[1]
    num_pop = compartment_sizes.sum()
    
    # Apply the infection process
    force_of_infection = contact_rate * num_inf / num_pop  # Frequency-dependent
    infection_flow_rate = force_of_infection * num_sus
    flow_rates[0] -= infection_flow_rate
    flow_rates[1] += infection_flow_rate

    # Infectious take some time to recover
    num_inf = compartment_sizes[1]
    recovery_flow_rate = recovery_rate * num_inf
    flow_rates[1] -= recovery_flow_rate
    flow_rates[2] += recovery_flow_rate
    
    # Add an infection-specific death flow to the I compartment
    num_inf = compartment_sizes[1]
    death_flow_rate = num_inf * death_rate
    flow_rates[1] -= death_flow_rate
    
    # Calculate compartment sizes at next timestep given flowrates
    outputs[t_idx] = compartment_sizes + flow_rates * TIMESTEP  
    
compartments = (
    "susceptible",
    "infectious",
    "recovered",
)
outputs_df = pd.DataFrame(outputs, columns=compartments)
outputs_df.plot()